In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Installed to a local directory and now I keep adding the notebook to this work

!pip install tensorrt onnxruntime # --quiet
# !pip install --upgrade tensorrt onnxruntime --target=/kaggle/working/mysitepackages

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.6.0-py2.py3-none-any.whl size=16337 sha256=f94c6de65adf0c7406ff3136d2d5eb6b3757d779a7d379d801e3341a980dd3c9
  Stored in directory: /root/.cache/pip/wheels/2a/97/95/8fdbe17369eed28ee9903523b94e130023ae58edcf1c904e5a
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.6.0-py2.py3-none-any.whl size=17551 sha256=1b359eb40cfee7ee00f55fea087936daad074006e67985c815c5faf357e9da3b
  Stored in directory: /root/.cache/pip/wheels/78/db/ca/0e81d122890b48aaa016992e062a1eb16dc3d00ac546156517
Successfully built tensorrt tensorrt-cu12


In [3]:
# imports
import torch
import transformers
# from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.onnx import OnnxConfig

from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import onnx

In [4]:
import onnxruntime as ort
import tensorrt as trt

# Covert to onnx

In [5]:
# Important to do this step in Kaggle only while creating :/
! touch tiny_llama.onnx

In [6]:
# Load model and tokenizer
device='cpu' # 'cuda'

model_path = "/kaggle/input/m/mambagetout/tinyllama/pytorch/default/1/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token # added because of generate thing

model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [7]:
from transformers import StaticCache

In [8]:
# Create dummy inputs
batch_size = 1
sequence_length = 256  # Reduced for testing

# Get model configuration
num_layers = model.config.num_hidden_layers
num_attention_heads = model.config.num_attention_heads
hidden_size = model.config.hidden_size
head_dim = hidden_size // num_attention_heads

In [9]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, input_ids, attention_mask, position_ids):
        # Initialize static cache
        past_key_values = StaticCache(
            config=self.model.config,
            batch_size=batch_size,
            max_cache_len= sequence_length, # 32,  # Match the sequence length
            device=input_ids.device,
            dtype=self.model.dtype
        )
        
        # Generate cache position
        cache_position = torch.arange(sequence_length, device=input_ids.device)
        
        # Forward pass with static cache
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            cache_position=cache_position, # Can keep None also? TRYING FOR NOW
            use_cache=True,
            return_dict=False
        )
        
        return outputs[0]  # Return only logits for simplicity

# Wrap the model, keeping on gpu is essestial else 
# error two devices found cuda and cuda:0 error
# wrapped_model = ModelWrapper(model).cuda()
wrapped_model = ModelWrapper(model).to(device)

# Input tensors
dummy_input_ids = torch.ones(batch_size, sequence_length, dtype=torch.long, device=device)
dummy_attention_mask = torch.ones(batch_size, sequence_length, dtype=torch.long, device=device)
dummy_position_ids = torch.arange(sequence_length, device=device).unsqueeze(0).repeat(batch_size, 1)

# Define dynamic axes for variable sequence lengths
dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence_length'},
    'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
    'position_ids': {0: 'batch_size', 1: 'sequence_length'},
}

In [10]:
# Export the model
# using cpu

torch.onnx.export(
    wrapped_model,
    (
        dummy_input_ids,
        dummy_attention_mask,
        dummy_position_ids,
    ),
    # '/kaggle/working/tiny_llama.onnx',
    '/kaggle/working/tiny_llama.onnx',
    input_names=['input_ids', 'attention_mask', 'position_ids'],
    output_names=['logits'],
    dynamic_axes=dynamic_axes,
    opset_version=18, # DO NOT, I REPEAT DO NOT USE OPSET_VERSION 15. FULL PAIN IT ISSSS.
    # do_constant_folding=False, # True, # CHECKING ANOTHER OPTION because later error. Depreciated only!
    verbose=False,

    # Extra params because of export exceeding size issue
    export_params=True, # IMP
    # export_type=torch.onnx.ExportTypes.PROTOBUF_FILE,
    
    # Specify external data parameters
    # storage_format=torch.onnx.FileFormat.EXTERNAL,
    # external_data_format=torch.onnx.FileFormat.PROTOBUF,
)

/opt/conda/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py:97: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


## **load the model**

In [11]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        if '.onnx' in filename:
            print(os.path.join(dirname, filename))

/kaggle/working/tiny_llama.onnx


In [12]:
onnx_model = onnx.load("/kaggle/working/tiny_llama.onnx")

In [13]:
import sys
sys.getsizeof(onnx_model)

96

In [14]:
# onnx.checker.check_model(onnx_model)

# Might give
# ValueError: Message onnx.ModelProto exceeds maximum protobuf size of 2GB: 4402590179
# But this is protobuf's mistake not ours xD

## Run Inference using the ort InferenceSession

In [15]:
ort_session = ort.InferenceSession(
    "/kaggle/working/tiny_llama.onnx", 
    providers=["CPUExecutionProvider"]
)

In [16]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {
    ort_session.get_inputs()[0].name: to_numpy(dummy_input_ids),
    ort_session.get_inputs()[1].name: to_numpy(dummy_attention_mask),
    ort_session.get_inputs()[2].name: to_numpy(dummy_position_ids)
}

ort_outs = ort_session.run(None, ort_inputs)

In [17]:
type(ort_outs), len(ort_outs), ort_outs[0].shape

(list, 1, (1, 256, 32000))

In [18]:
# check if outputs are close
np.testing.assert_allclose(
    to_numpy(wrapped_model(dummy_input_ids, dummy_attention_mask, dummy_position_ids)), 
    ort_outs[0], 
    rtol=1e-03, 
    atol=1e-04
)
# throws assertion error when atol is 1e-05
# So I think this is fair enough

## this works, so now work with text inputs

### Check the inputs

In [19]:
print(f'dummy_input_ids = {dummy_input_ids} \n dummy_attention_mask = {dummy_attention_mask} \n dummy_position_ids = {dummy_position_ids}')

dummy_input_ids = tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) 
 dummy_attention_mask = tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       

In [20]:
def prepare_inputs(text_input, tokenizer, target_length=256, device='cuda'):
    """
    Prepare inputs by padding/truncating to target length
    """
    # Tokenize the input
    encoded = tokenizer.encode(text_input, add_special_tokens=True, return_tensors="pt")
    input_ids = encoded[0]  # Remove batch dimension
    
    # Get the actual sequence length
    seq_len = len(input_ids)
    
    if seq_len > target_length:
        # Truncate if longer than target_length
        input_ids = input_ids[:target_length]
        seq_len = target_length
    
    # Pad input_ids
    padding_length = target_length - seq_len
    padding_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    input_ids = torch.cat([
        input_ids,
        torch.full((padding_length,), padding_id, dtype=torch.long)
    ])
    
    # Create attention mask (1 for real tokens, 0 for padding)
    attention_mask = torch.cat([
        torch.ones(seq_len, dtype=torch.long),  # Explicitly set dtype to long
        torch.zeros(padding_length, dtype=torch.long)  # Explicitly set dtype to long
    ])
    
    # Create position ids
    # position_ids = torch.arange(target_length)
    position_ids = torch.arange(target_length, dtype=torch.long)  # Explicitly set dtype to long

    
    # Add batch dimension and move to device
    input_ids = input_ids.unsqueeze(0).to(device)
    attention_mask = attention_mask.unsqueeze(0).to(device)
    position_ids = position_ids.unsqueeze(0).to(device)
    
    return input_ids, attention_mask, position_ids

In [21]:
def make_both_inferences(text_input: str):

    input_ids, attention_mask, position_ids = prepare_inputs(
        text_input, 
        tokenizer, 
        target_length=256,
        device=device
    )

    wrapped_model_outputs = to_numpy(wrapped_model(input_ids, attention_mask, position_ids))

    ort_inputs = { 
        ort_session.get_inputs()[0].name: to_numpy(input_ids),
        ort_session.get_inputs()[1].name: to_numpy(attention_mask),
        ort_session.get_inputs()[2].name: to_numpy(position_ids)
    }
    ort_outs = ort_session.run(None, ort_inputs)

    return wrapped_model_outputs, ort_outs

In [22]:
wrapped_model_outputs, ort_outs = make_both_inferences(text_input ='hi there my work is computer science and I like')

In [23]:
wrapped_model_outputs, ort_outs

(array([[[ -7.396678  ,  -7.3966794 ,   8.475743  , ...,  -4.358731  ,
           -7.0164227 ,  -0.75330424],
         [ -6.5529804 ,  -6.5529814 ,   7.802591  , ...,  -4.382573  ,
           -6.0285573 ,  -0.10475935],
         [ -6.6338015 ,  -6.6337953 ,   8.605309  , ...,  -4.369956  ,
           -5.029665  ,   0.3729149 ],
         ...,
         [-10.389168  , -10.38916   ,   4.1876626 , ...,  -5.8836117 ,
           -7.9302125 ,  -3.226597  ],
         [-10.757662  , -10.757653  ,   3.5645452 , ...,  -5.9962487 ,
           -7.8819776 ,  -3.4362864 ],
         [-11.012067  , -11.012054  ,   3.273897  , ...,  -6.005757  ,
           -7.9752884 ,  -3.61273   ]]], dtype=float32),
 [array([[[ -7.3966737 ,  -7.3966737 ,   8.475735  , ...,  -4.3587313 ,
            -7.01641   ,  -0.7532996 ],
          [ -6.552985  ,  -6.5529885 ,   7.8025846 , ...,  -4.3825774 ,
            -6.028565  ,  -0.10476124],
          [ -6.6337914 ,  -6.633787  ,   8.605286  , ...,  -4.3699517 ,
            

## The token ids converted back to text aren't beutiful though. 

In [24]:
 # Convert logits to token IDs (if necessary)
wrapped_token_ids = torch.argmax(torch.from_numpy(wrapped_model_outputs), dim=-1)
ort_token_ids = torch.argmax(torch.from_numpy(ort_outs[0]), dim=-1)  # Assuming first output is relevant

# Decode token IDs back to text
wrapped_text_output = tokenizer.decode(wrapped_token_ids[0], skip_special_tokens=True)
ort_text_output = tokenizer.decode(ort_token_ids[0], skip_special_tokens=True)
wrapped_text_output, ort_text_output

('amedag. my. my. computer computer am I I I I I I I I I I computer I I I I computer computer computer I I I I I I I I I I computer computer computer computer I I I I I I I I computer computer I I I I I I I I I I I I computer computer computer I I I computer computer I I I I I I I I I I computer computer computer I I I I I I I I I I I I I I I I I I I I I I I computer I I I I I I I I I I I I I I I I I computer computer I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I computer computer I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I computer I I I I computer I I I I I I I I I I I I I I I I I I I I I I I',
 'amedag. my. my. computer computer am I I I I I I I I I I computer I I I I computer computer computer I I I I I I I I I I computer computer computer computer I I I I I I I I computer computer I I I I I I I I I I I I computer computer computer I I I computer computer I I I I I I I I I I comp